**Part 1: Preparing the training data**


In [0]:
# the required imports for training and fitting the model
import tensorflow as tf
import numpy as np
import os

In [0]:
# Process the file and produce the training dataset
# read the file
filename = 'the-office.txt'
file = open(filename,'rb')
text = file.read().decode('utf8')
file.close()

# vectorize the text
vocab = sorted(set(text)) # unique characters in the file
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# divide text into example input sequences of length:seq_length
# corresponding example output sequences are same length excepted shifted one character
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) # create training examples
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) # convert individual characters to sequences

# for each sequence, form the input and output text 
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
dataset = sequences.map(split_input_target)

# create training batches by shuffling the data and packing it into batches
batch_size = 64
buffer_size = 10000
dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size, drop_remainder=True)

**Part 2: Build and train the model**

In [0]:
# Build the model
vocab_size = len(vocab) 
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
    return model

In [4]:
model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=batch_size)
model.summary()

# get output distribution from model
for input_example_batch, output_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch) 

# to get actual character indices, sample from output distribution
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
    
model.compile(optimizer='adam', loss=loss) # attach optimizer and loss function 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           22784     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 89)            91225     
Total params: 13,753,689
Trainable params: 13,753,689
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Train the model
# ensure checkpoints are saved during training
!mkdir -p training
checkpoint_dir = './training' # directory where checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir,"checkpoint_{epoch}") # name of checkpoint files
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

epochs = 50
history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

In [0]:
# Rebuild model and restore context from last checkpoint
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint("./training"))
model.build(tf.TensorShape([1, None]))
model.summary()

# Save the weights
model.save_weights('./training/training_data')

**Part 3: Generate scripts**

In [0]:
# Generate text
def generate_text(model,start_string, num = 1000, temp = 0.50):
    # converting start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    # Empty string to store our results
    text_generated = []
    model.reset_states() # reset context from previous use
    for i in range(num):
        predictions = model(input_eval)
        # remove batch dimension
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temp
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        # pass predicted character as next input to the model along with the previous (hidden) state
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string="JIM:\n", num=1000, temp=0.25))

**Part 4 (Optional): Exporting the model**

*This would require restarting the runtime and reimporting an earlier version of Tensorflow in order to use some of the deprecated functionality. Some of the previous cells may need to be rerun to prevent errors.*

In [0]:
%tensorflow_version 1.1x

import tensorflow as tf
import numpy as np
import os

tf.enable_eager_execution()
tf.keras.backend.clear_session()

In [0]:
# Create a new model instance
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Restore the weights
model.load_weights('./training/training_data')
model.build(tf.TensorShape([1, None]))
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)) # attach optimizer and loss function

# Save the model
!mkdir -p saved_model
tf.keras.experimental.export_saved_model(model,'python_model')

tf.compat.v1.disable_eager_execution()
tf.keras.backend.clear_session()
new_model=tf.keras.experimental.load_from_saved_model('python_model')

# Check its architecture
new_model.summary()

In [0]:
# save the model to implement on the web using tensorflow.js
!pip install tensorflowjs
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(new_model,'./javascript_model/model')
!zip -r javascript_model.zip javascript_model